In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.restoration import denoise_wavelet, estimate_sigma
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error as mse
from skimage import img_as_float, img_as_ubyte
from skimage.util import random_noise
import tensorflow as tf
import time

In [ ]:
def add_noise_to_image(image, noise_type='gaussian', amount=0.05):
    """Add noise to an image for testing denoising methods."""
    float_img = img_as_float(image)
    noisy_img = random_noise(float_img, mode=noise_type, var=amount)
    return noisy_img, float_img

def median_filter_denoise(noisy_img, kernel_size=3):
    """Apply median filter for denoising."""
    denoised = cv2.medianBlur(img_as_ubyte(noisy_img), kernel_size)
    return img_as_float(denoised)

def wavelet_denoise(noisy_img, wavelet='db1', mode='soft'):
    """Apply wavelet transform for denoising."""
    sigma_est = estimate_sigma(noisy_img, average_sigmas=True)
    return denoise_wavelet(noisy_img, sigma=sigma_est, wavelet=wavelet, mode=mode,
                         rescale_sigma=True)

In [ ]:
# Simple implementation of Noise2Void concept
def noise2void_denoise(noisy_img, patch_size=5):
    """Simplified version of Noise2Void using basic masking approach."""
    h, w = noisy_img.shape[:2]
    result = np.copy(noisy_img)

    # For simplicity, we'll use a moving window approach with median values
    for i in range(patch_size, h-patch_size):
        for j in range(patch_size, w-patch_size):
            # Take surrounding patch but exclude center pixel
            patch = noisy_img[i-patch_size:i+patch_size+1, j-patch_size:j+patch_size+1].copy()
            center = patch[patch_size, patch_size].copy()
            patch[patch_size, patch_size] = 0  # Mask center

            # Use median of neighborhood as estimate
            median_val = np.median(patch[patch != 0])
            result[i, j] = median_val

    return result

In [ ]:
def compare_denoising_methods(image_path, noise_type='gaussian', noise_amount=0.05):
    """Compare different denoising methods and calculate metrics."""
    # Load image and convert to grayscale if needed
    original = cv2.imread(image_path)
    if original is None:
        raise ValueError(f"Could not read image at {image_path}")

    if len(original.shape) == 3 and original.shape[2] == 3:
        original = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)

    # Normalize to [0, 1] range
    original_norm = img_as_float(original)

    # Add noise
    noisy_img, _ = add_noise_to_image(original_norm, noise_type, noise_amount)

    # Apply denoising methods
    start_time = time.time()
    median_denoised = median_filter_denoise(noisy_img)
    median_time = time.time() - start_time

    start_time = time.time()
    wavelet_denoised = wavelet_denoise(noisy_img)
    wavelet_time = time.time() - start_time

    start_time = time.time()
    n2v_denoised = noise2void_denoise(noisy_img)
    n2v_time = time.time() - start_time
    # Calculate metrics
    results = {
        'Method': ['Median Filter', 'Wavelet Denoising', 'Noise2Void'],
        'PSNR': [
            psnr(original_norm, median_denoised),
            psnr(original_norm, wavelet_denoised),
            psnr(original_norm, n2v_denoised)
        ],
        'SSIM': [
            # Specify data_range for ssim calculations
            ssim(original_norm, median_denoised, data_range=1.0),  # Assuming data range is [0, 1]
            ssim(original_norm, wavelet_denoised, data_range=1.0),  # Assuming data range is [0, 1]
            ssim(original_norm, n2v_denoised, data_range=1.0)  # Assuming data range is [0, 1]
        ],
        'MSE': [
            mse(original_norm, median_denoised),
            mse(original_norm, wavelet_denoised),
            mse(original_norm, n2v_denoised)
        ],
        'Time (s)': [median_time, wavelet_time, n2v_time]
    }

    # Display results
    plt.figure(figsize=(20, 10))

    plt.subplot(2, 3, 1)
    plt.imshow(original_norm, cmap='gray')
    plt.title('Original')
    plt.axis('off')

    plt.subplot(2, 3, 2)
    plt.imshow(noisy_img, cmap='gray')
    plt.title(f'Noisy ({noise_type})')
    plt.axis('off')

    plt.subplot(2, 3, 3)
    plt.imshow(median_denoised, cmap='gray')
    plt.title(f'Median Filter\nPSNR: {results["PSNR"][0]:.2f}, SSIM: {results["SSIM"][0]:.2f}')
    plt.axis('off')

    plt.subplot(2, 3, 4)
    plt.imshow(wavelet_denoised, cmap='gray')
    plt.title(f'Wavelet Denoising\nPSNR: {results["PSNR"][1]:.2f}, SSIM: {results["SSIM"][1]:.2f}')
    plt.axis('off')

    plt.subplot(2, 3, 5)
    plt.imshow(n2v_denoised, cmap='gray')
    plt.title(f'Noise2Void\nPSNR: {results["PSNR"][2]:.2f}, SSIM: {results["SSIM"][2]:.2f}')
    plt.axis('off')

    plt.tight_layout()
    plt.savefig('denoising_comparison.png')
    plt.close()

    # Print metrics table
    print("\nDenoising Methods Comparison:")
    print("-" * 80)
    print(f"{'Method':<20} {'PSNR':<10} {'SSIM':<10} {'MSE':<10} {'Time (s)':<10}")
    print("-" * 80)
    for i in range(len(results['Method'])):
        print(f"{results['Method'][i]:<20} {results['PSNR'][i]:<10.4f} {results['SSIM'][i]:<10.4f} {results['MSE'][i]:<10.4f} {results['Time (s)'][i]:<10.4f}")

    return results, {
        'original': original_norm,
        'noisy': noisy_img,
        'median': median_denoised,
        'wavelet': wavelet_denoised,
        'n2v': n2v_denoised
    }

In [ ]:
def extract_frames(video_path, output_folder='extracted_frames', frame_step=1):
    """Extract frames from a video file and save them as images."""
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video file {video_path}")

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps

    print(f"Video Information:")
    print(f"FPS: {fps}")
    print(f"Frame Count: {frame_count}")
    print(f"Duration: {duration:.2f} seconds")

    # Extract frames
    count = 0
    frames_saved = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Save frame at the specified interval
        if count % frame_step == 0:
            frame_filename = os.path.join(output_folder, f"frame_{frames_saved:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
            frames_saved += 1

        count += 1

    cap.release()
    print(f"Extracted {frames_saved} frames to {output_folder}")
    return output_folder, fps


In [ ]:
"""
PART 3: PROCESS VIDEO FRAMES AND CREATE NEW VIDEOS
"""
def process_video_frames(video_path, output_folder='processed_videos'):
    """Process video frames with different techniques and create new videos."""
    # Create output folders
    os.makedirs(output_folder, exist_ok=True)
    frames_folder = os.path.join(output_folder, 'frames')
    os.makedirs(frames_folder, exist_ok=True)

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video file {video_path}")

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Define video writers for each processing technique
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    adaptive_thresh_video = cv2.VideoWriter(os.path.join(output_folder, 'adaptive_threshold.mp4'),
                                           fourcc, fps, (width, height), False)
    gaussian_video = cv2.VideoWriter(os.path.join(output_folder, 'gaussian_smoothing.mp4'),
                                    fourcc, fps, (width, height))
    canny_video = cv2.VideoWriter(os.path.join(output_folder, 'canny_edge.mp4'),
                                 fourcc, fps, (width, height), False)
    bitwise_not_video = cv2.VideoWriter(os.path.join(output_folder, 'bitwise_not.mp4'),
                                       fourcc, fps, (width, height))

    # Process frames
    frame_set = []  # Store some frames for the collage
    frame_step = max(1, frame_count // 25)  # Store about 25 frames for collage

    count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert to grayscale for some operations
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # 1. Adaptive Thresholding
        adaptive_thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                               cv2.THRESH_BINARY, 11, 2)

        # 2. Gaussian Smoothing
        gaussian = cv2.GaussianBlur(frame, (15, 15), 0)

        # 3. Canny Edge Detection
        canny = cv2.Canny(gray, 100, 200)

        # 4. Bitwise Not
        bitwise_not = cv2.bitwise_not(frame)

        # Write frames to videos
        adaptive_thresh_video.write(adaptive_thresh)
        gaussian_video.write(gaussian)
        canny_video.write(canny)
        bitwise_not_video.write(bitwise_not)

        # Save frames for collage
        if count % frame_step == 0:
            # Save original and processed frames
            cv2.imwrite(os.path.join(frames_folder, f"original_{count:04d}.jpg"), frame)
            cv2.imwrite(os.path.join(frames_folder, f"adaptive_{count:04d}.jpg"), adaptive_thresh)
            cv2.imwrite(os.path.join(frames_folder, f"gaussian_{count:04d}.jpg"), gaussian)
            cv2.imwrite(os.path.join(frames_folder, f"canny_{count:04d}.jpg"), canny)
            cv2.imwrite(os.path.join(frames_folder, f"bitwise_{count:04d}.jpg"), bitwise_not)

            # Store for collage
            frame_set.append({
                'original': frame.copy(),
                'adaptive': adaptive_thresh.copy(),
                'gaussian': gaussian.copy(),
                'canny': canny.copy(),
                'bitwise': bitwise_not.copy()
            })

        count += 1

    # Release resources
    cap.release()
    adaptive_thresh_video.release()
    gaussian_video.release()
    canny_video.release()
    bitwise_not_video.release()

    # Create collage
    create_collage(frame_set, os.path.join(output_folder, 'video_processing_collage.jpg'))

    print(f"Processed {count} frames and created videos in {output_folder}")
    return output_folder

In [ ]:
def create_collage(frame_set, output_path):
    """Create a collage from the processed frames."""
    if not frame_set:
        print("No frames to create collage")
        return

    # Determine dimensions
    num_frames = len(frame_set)
    num_types = 5  # original, adaptive, gaussian, canny, bitwise

    # Let's make a grid with rows = num_types and cols = min(5, num_frames)
    cols = min(5, num_frames)
    rows = num_types

    # Get frame dimensions from first frame
    f_height, f_width = frame_set[0]['original'].shape[:2]

    # Create blank canvas
    collage_width = cols * f_width
    collage_height = rows * f_height
    collage = np.zeros((collage_height, collage_width, 3), dtype=np.uint8)

    # Fill the collage
    for i, frame_dict in enumerate(frame_set[:cols]):
        # Original
        collage[0:f_height, i*f_width:(i+1)*f_width] = frame_dict['original']

        # Adaptive threshold (convert to BGR for display)
        adaptive_bgr = cv2.cvtColor(frame_dict['adaptive'], cv2.COLOR_GRAY2BGR)
        collage[f_height:2*f_height, i*f_width:(i+1)*f_width] = adaptive_bgr

        # Gaussian blur
        collage[2*f_height:3*f_height, i*f_width:(i+1)*f_width] = frame_dict['gaussian']

        # Canny edges (convert to BGR for display)
        canny_bgr = cv2.cvtColor(frame_dict['canny'], cv2.COLOR_GRAY2BGR)
        collage[3*f_height:4*f_height, i*f_width:(i+1)*f_width] = canny_bgr

        # Bitwise not
        collage[4*f_height:5*f_height, i*f_width:(i+1)*f_width] = frame_dict['bitwise']

    # Add labels
    font = cv2.FONT_HERSHEY_SIMPLEX
    labels = ['Original', 'Adaptive Threshold', 'Gaussian Blur', 'Canny Edge', 'Bitwise Not']

    for i, label in enumerate(labels):
        y_pos = i * f_height + 30
        cv2.putText(collage, label, (10, y_pos), font, 1, (255, 255, 255), 2)

    # Save the collage
    cv2.imwrite(output_path, collage)
    print(f"Collage saved to {output_path}")


In [ ]:
!pip install PyWavelets
!pip install opencv-python numpy matplotlib scikit-image tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 61.3 MB/s eta 0:00:00


In [ ]:
"""
MAIN FUNCTION TO RUN ALL TASKS
"""
def main():
    # TASK 1: Image Denoising Comparison
    print("\n" + "="*50)
    print("TASK 1: IMAGE DENOISING COMPARISON")
    print("="*50)

    # You can replace this with the path to your own image
    # If you don't have an image, uncomment the code below to create a sample image
    """
    # Create a sample image if needed
    sample_img = np.zeros((300, 300), dtype=np.uint8)
    cv2.circle(sample_img, (150, 150), 100, 255, -1)
    cv2.rectangle(sample_img, (50, 50), (250, 250), 128, 3)
    cv2.imwrite('sample_image.jpg', sample_img)
    image_path = 'sample_image.jpg'
    """

    # Replace with your image path
    image_path = '/content/denoise.png'

    # Uncomment the line below and provide your image path to run the denoising comparison
    results, images = compare_denoising_methods(image_path, noise_type='gaussian', noise_amount=0.05)

    print("\n" + "="*50)
    print("TASK 2: EXTRACT FRAMES FROM VIDEO")
    print("="*50)

    # Replace with your video path
    video_path = '/content/sample.mp4'

    # Uncomment the line below and provide your video path to extract frames
    frames_folder, fps = extract_frames(video_path, output_folder='extracted_frames', frame_step=5)

    print("\n" + "="*50)
    print("TASK 3: PROCESS VIDEO FRAMES")
    print("="*50)

    # Use the same video path or specify a different one
    # Uncomment the line below and provide your video path to process the video
    output_folder = process_video_frames(video_path, output_folder='processed_videos')

    print("\nAll tasks completed!")

if __name__ == "__main__":
    main()


TASK 1: IMAGE DENOISING COMPARISON

Denoising Methods Comparison:
--------------------------------------------------------------------------------
Method               PSNR       SSIM       MSE        Time (s)  
--------------------------------------------------------------------------------
Median Filter        14.0352    0.2165     0.0395     0.0007    
Wavelet Denoising    14.4659    0.1517     0.0358     0.0057    
Noise2Void           14.0442    0.1116     0.0394     2.1805    

TASK 2: EXTRACT FRAMES FROM VIDEO
Video Information:
FPS: 25.0
Frame Count: 132
Duration: 5.28 seconds
Extracted 27 frames to extracted_frames

TASK 3: PROCESS VIDEO FRAMES
Collage saved to processed_videos/video_processing_collage.jpg
Processed 132 frames and created videos in processed_videos

All tasks completed!
